In [1]:
import sys
import os

# Go up one directory (from notebooks/ to OLIST/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to the Python path
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project root added to path: {project_root}")

Project root added to path: d:\My_Projects\OLIST\python


In [ ]:
from src.utils import fetch_data_from_bq
import pandas as pd
from src import sql_queries as q
import pandas as pd
import numpy as np



In [32]:
df = fetch_data_from_bq(q.GET_PRODUCTS)
df

Query successful. Scanned 0.000 MB.


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,a0ab96e461d74537772b84950f26a257,climatizacao,41,717,1,1050,18,7,8
1,20ae7c024ede613f47e0d2f23f461493,telefonia_fixa,25,455,1,330,17,11,9
2,4d7585daba2f8b3ed7f87447908b4237,telefonia_fixa,53,897,2,300,15,8,9
3,ad7aebed205805125489f8a89819b24b,construcao_ferramentas_ferramentas,41,2526,2,1150,22,10,9
4,980ecbcc15fe174ec1e5757c4d75b1bf,agro_industria_e_comercio,48,157,1,250,17,3,10
...,...,...,...,...,...,...,...,...,...
32946,e7e460583f2b78939dee0f8cd9a619ed,utilidades_domesticas,59,586,6,12250,80,10,76
32947,b92268ad64d3947c80fddbc3a82de845,utilidades_domesticas,55,134,1,8850,39,12,78
32948,4047f50df2141e2ab6c747b16b6a7bf6,utilidades_domesticas,40,564,3,30000,90,30,80
32949,66a4ac0f9c73e92afc9cb7566977ef83,utilidades_domesticas,58,547,5,1000,18,12,82


In [12]:
df.count()
n_rows = len(df)
n_cols = len(df.columns)
print(f"DataFrame has {n_rows} rows and {n_cols} columns.")


DataFrame has 1000163 rows and 5 columns.


In [20]:
n_dup_rows = len(df.duplicated(keep=False))
print(f"DataFrame has {n_dup_rows} duplicated rows.")

DataFrame has 1000163 duplicated rows.


In [5]:


def perform_data_qc(df, df_name="DataFrame"):
    """
    Performs a two-step Quality Control check on a pandas DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to check.
        df_name (str): The name of the DataFrame for clear output.
    """
    print("="*80)
    print(f"        *** Quality Control Report for {df_name} ***")

    # --- Table Shape and Duplicates ---
    
    print("\n### Table Information Overview ###")
    n_rows = len(df)
    n_cols = len(df.columns)
    print(f"Total Rows: {n_rows}")
    print(f"Table Name: {df_name}")
    print(f"Total Columns: {n_cols}")

    # Calculate total duplicated rows (where the entire row is a duplicate)
    n_dup_rows = df.duplicated().sum() 
    print(f"Total Duplicated Rows: {n_dup_rows}")
    print("-" * 80)

    # --- Nulls and Column Information Overview ---
    print("\n#### Null Value Analysis (For Each Columns) ####")
    
    # Calculate null counts and percentages
    null_counts = df.isnull().sum()
    null_percents = (null_counts / n_rows) * 100
    
    # Create a summary DataFrame
    null_summary = pd.DataFrame({
        'Dtype': df.dtypes,
        'Null Count': null_counts,
        'Null Percent': null_percents.round(2).astype(str) + '%'
    }).sort_values(by='Null Count', ascending=False)

    print(null_summary)
    print("-" * 80)

    # --- Basic Statistics for Numeric Columns ---
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    
    if numeric_cols:
        print("\n#### Basic Statistics (Numeric Columns) ####")
        stats = df[numeric_cols].agg(['min', 'median', 'max']).T
        print(stats)
    else:
        print("\n* No Numeric Columns Found")
    print("-" * 80)
    
    # --- Value Counts for Categorical Columns ---
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

    if categorical_cols:
        print("\n#### Value Counts (Categorical Columns) ####")
        for col in categorical_cols:
            print(f"\nColumn: {col}")
            # Display the top 5 most frequent values for brevity
            counts = df[col].value_counts(dropna=False)
            print(counts.head(3)) 
            if len(counts) > 3:
                print(f"... and {len(counts) - 3} more unique values.")
    else:
        print("\n* No Categorical/Object Columns Found")
    
    print("\n" + "="*80)


In [6]:
# --- List all DataFrames you want to process ---
dataframe_list = [ 
    q.GET_PRODUCTS, 
    q.GET_SELLERS
]

# --- Define corresponding names for the output report ---
dataframe_names = [

    "PRODUCTS", 
    "SELLERS"
]

# --- Loop through the list and run the QC function ---
for df_name, name in zip(dataframe_list, dataframe_names):
    perform_data_qc(fetch_data_from_bq(df_name), df_name=name)


BigQuery clients initialized.
Query successful. Scanned 0.000 MB.
        *** Quality Control Report for PRODUCTS ***

### Table Information Overview ###
Total Rows: 32951
Table Name: PRODUCTS
Total Columns: 9
Total Duplicated Rows: 0
--------------------------------------------------------------------------------

#### Null Value Analysis (For Each Columns) ####
                             Dtype  Null Count Null Percent
product_category_name       object         610        1.85%
product_description_lenght   Int64         610        1.85%
product_name_lenght          Int64         610        1.85%
product_photos_qty           Int64         610        1.85%
product_weight_g             Int64           2        0.01%
product_height_cm            Int64           2        0.01%
product_length_cm            Int64           2        0.01%
product_width_cm             Int64           2        0.01%
product_id                  object           0         0.0%
----------------------------------